In [1]:
###Reyes, Marcus Group 7

###CoE 197Z Project 1.1

import pandas as pd

import keras
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout,BatchNormalization
from keras import regularizers
from keras.optimizers import adam,sgd
from keras import optimizers

import numpy as np
from numpy import genfromtxt

from sklearn import preprocessing

import matplotlib.pyplot as plt

import string

#from sixfunctions import load_train, load_x_test, clean_data_with_mean
!pip install category_encoders

Using TensorFlow backend.


     |████████████████████████████████| 102kB 3.3MB/s 


#### Under the hood

In [0]:
import pandas as pd
import numpy as np

from numpy import genfromtxt
from sklearn import preprocessing

import matplotlib.pyplot as plt
import string
from keras import backend as K

from sklearn.preprocessing import OneHotEncoder


#Given the dataframe and the column it replaces all zero nonavailable values with the mean
def clean_data_with_mean(data, column):
    replace_map = {column:{0:np.nan}}
    data.replace(replace_map, inplace=True)

    mean = data[column].mean()
    data[column] = data[column].fillna(0)
    replace_map = {column:{0:mean}}
    data.replace(replace_map, inplace=True)
    return data

def date_to_cyclic(data):
    # For date_recorder, convert to three columns, 2 for cyclical day of year recorded and the year recorded
    # https://www.avanwyk.com/encoding-cyclical-features-for-deep-learning/
    data["date_recorded"] = pd.to_datetime(data["date_recorded"], format="%Y-%m-%d")
    data["day_of_year_recorded"] = data["date_recorded"].apply(lambda x: x.timetuple().tm_yday)

    data["year_recorded"] = data["date_recorded"].apply(lambda x: x.year)
    data["doy_recorded_sin"] = data["day_of_year_recorded"].apply(lambda x: np.sin(2 * np.pi * x/365.0))
    data["doy_recorded_cos"] = data["day_of_year_recorded"].apply(lambda x: np.cos(2 * np.pi * x/365.0))
    data = data.drop(["date_recorded", "day_of_year_recorded"], axis='columns')

    return data

def onehot_encode(train, test, cols):
    print("one-hot encoding:")
    from sklearn.preprocessing import OneHotEncoder
    for col in cols:
        print(f"\t{col}:", end='\t')

        train[col] = train[col].fillna("")
        test[col]  = test[col].fillna("")

        encoder = OneHotEncoder(sparse=False, categories='auto', handle_unknown='ignore')
        onehot_cols = encoder.fit_transform(train[col].to_numpy().reshape(-1,1))
        categories = encoder.categories_[0]
        col_names = [f"{col}_{cat}" for cat in categories]  # names of new onehot columns

        onehot_cols = pd.DataFrame(onehot_cols, columns=col_names)
        train = pd.concat([train.drop([col], axis='columns'), onehot_cols], axis='columns')

        onehot_cols = encoder.transform(test[col].to_numpy().reshape(-1,1))
        onehot_cols = pd.DataFrame(onehot_cols, columns=col_names)
        test = pd.concat([test.drop([col], axis='columns'), onehot_cols], axis='columns')
        
        print(str(len(categories)) + " categories")
    return train, test

def nominal_to_binary(train, test, cols):
    print("binary encoding:")
    #!pip install category_encoders
    from category_encoders import BinaryEncoder
    import math
    for col in cols:
        n_unique = len(train[col].unique())
        print('\t', col, end=" ")
        print('\t', n_unique, " unique values", " -> ", end='')
        print(math.ceil(math.log(n_unique, 2)), " columns")

        encoder = BinaryEncoder(verbose=1, cols=[col])
        train = encoder.fit_transform(train)
        test  = encoder.transform(test)

        i = 0
        while(f"col_{i}" in train.columns):
            rename_format = {f"col_{i}": f"{col}_col_{i}"}
            train = train.rename(columns=rename_format)
            test  = test.rename(columns=rename_format)
            i += 1
    return train, test

def preprocess_data(train, labels, test, drop_cols=None, unique_cols=None,
                    binary_cols=None, onehot_cols=None, scale_cols=None):
    # sets of columns
    if not drop_cols:
        drop_cols = {'id', 'num_private',
                    'quality_group',   # exact duplicate of quality
                    'recorded_by'}     # uniform value
    if not unique_cols:
        unique_cols = { 'funder',       # 1898 unique values
                        'ward',         # 2092 unique values
                        'installer',    # 2146 unique values
                        'scheme_name',  # 2697 unique values
                        'subvillage',   # 19288 unique values
                        'wpt_name' }    # 37400 unique values
    if not binary_cols:    
        binary_cols = {"permit", "public_meeting"}
    object_cols = set(train.select_dtypes(include=['object']).columns) - drop_cols
    if not onehot_cols:
        onehot_cols = ((((object_cols | {'region_code', 'district_code'})
                    - {'date_recorded', 'permit', 'public_meeting'})
                    - unique_cols) - drop_cols)
    if not scale_cols:
        scale_cols  = (((((set(train.select_dtypes(include=['number']).columns)
                    - onehot_cols) - unique_cols) - binary_cols) - drop_cols)
                    | {"year_recorded"})

    # Drop irrelevant/redundant columns
    print(f"dropping: {drop_cols}")
    train = train.drop(drop_cols, axis='columns')
    test  = test.drop(drop_cols, axis='columns')

    # Onehot Encoding (train labels)
    labels = labels.pop('status_group').values
    labels = pd.get_dummies(labels)

    # Remove NaN's
    print("removing NaN's:")
    for col in binary_cols:
        print("\t", col)
        train[col] = train[col].fillna(False).astype('float64')
        test[col] = test[col].fillna(False).astype('float64')

    # Replace 0's with mean
    print("replacing 0's with mean:")
    for col in {"population", "amount_tsh", "construction_year"}:
        print("\t", col)
        train = clean_data_with_mean(train,col)
        test  = clean_data_with_mean(test,col)
    
    # Cyclic Encoding
    print("cyclic encoding:\n\tdate_recorded -> (doy_recorded_sin, doy_recorded_cos), year_recorded")
    train = date_to_cyclic(train)
    test = date_to_cyclic(test)

    # Normalization
    print("Normalization:")
    from sklearn.preprocessing import MinMaxScaler, StandardScaler
    scaler = StandardScaler()
    for col in scale_cols:
        print(f"\t{col} [{min(train[col]), max(train[col])}]", end=" -> ")
        train[col] = scaler.fit_transform(train[col].to_numpy().reshape(-1,1))
        test[col]  = scaler.transform(test[col].to_numpy().reshape(-1,1))
        print(f"[{min(train[col]), max(train[col])}]")

    # Onehot Encoding
    train, test = onehot_encode(train, test, onehot_cols)

    # Binary Encoding
    print("Binary Encoding:")
    train, test = nominal_to_binary(train, test, unique_cols)

    return train, labels, test

#### execution of preprocessing

In [34]:
train   = pd.read_csv("train_set_values.csv")
labels  = pd.read_csv("train_set_labels.csv")
test    = pd.read_csv("test_values.csv")

train, labels, test = preprocess_data(train, labels, test)

dropping: {'num_private', 'quality_group', 'id', 'recorded_by'}
removing NaN's:
	 public_meeting
	 permit
replacing 0's with mean:
	 amount_tsh
	 population
	 construction_year
cyclic encoding:
	date_recorded -> (doy_recorded_sin, doy_recorded_cos), year_recorded
Normalization:
	amount_tsh [(0.2, 350000.0)] -> [(-0.3590986010964154, 117.97090065790498)]
	gps_height [(-90, 2770)] -> [(-1.0940495369292693, 3.0322765453176466)]
	latitude [(-11.64944018, -2e-08)] -> [(-2.0174538540773703, 1.9368783817775395)]
	year_recorded [(2002, 2013)] -> [(-10.348549589913485, 1.1247289744119464)]
	population [(1.0, 30500.0)] -> [(-0.6199883131902885, 66.89122175181514)]
	construction_year [(1960.0, 2013.0)] -> [(-3.6574406124580006, 1.6079677259760634)]
	longitude [(0.0, 40.34519307)] -> [(-5.188894889753043, 0.9543790152566973)]
one-hot encoding:
	payment_type:	7 categories
	source_type:	7 categories
	payment:	7 categories
	quantity:	5 categories
	region_code:	27 categories
	source_class:	3 categorie

In [0]:
x = train.to_numpy()
y = labels.to_numpy()

#### Old

In [0]:
do_not_include = {'quality_group',      # exact duplicate of 'quality'
                  'recorded_by',        # uniform value

                  'wpt_name',           #unique
                  'subvillage',         #dup of loc, numerous
                  'scheme_name',        #messy numerous
                  'payment',            #duplicate of payment type
                  'quantity_group',     #Dup of quantity
                  'waterpoint_type_group',#dup of wtptype less data
                  'source_type',        #dup of source
                  'extraction_type',    #dup of extr_type_group
                  #'extraction_type_group',
                  'region'} - unique_cols#enc in regcode

#These are sort of ordinal data
do_not_one_hot = {'id','gps_height','doy_recorded_sin','doy_recorded_cos',
                  'longitude','latitude','population','amount_tsh'} | unique_cols

clean_up = {'population','amount_tsh'}

#Tentatively do not include.
do_not_include_tent = {'funder','installer','ward','lga'} - unique_cols

#Don't know what it means 
do_not_include_temp = {'num_private'} - unique_cols


x, train_col,y = load_train("train_set_values.csv","train_set_labels.csv",
                            do_not_include, do_not_one_hot,
                            clean_up, do_not_include_tent, do_not_include_temp,
                            unique_cols)

#### Model

In [37]:
###Model

hidden = 1024
dropout = 0.4

(trash, input_dim) = x.shape

activation = 'relu'

model = Sequential()


model.add(Dense(hidden, input_dim = input_dim))
model.add(Dropout(dropout))
model.add(Activation(activation))

model.add(Dense(hidden, input_dim = input_dim))
model.add(Dropout(dropout))
model.add(Activation(activation))

model.add(Dense(hidden, input_dim = input_dim))
model.add(Dropout(dropout))
model.add(Activation(activation))


model.add(Dense(3,input_dim = hidden))
model.add(Activation('softmax'))

#model.summary()

model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

#history = model.fit(x, y, epochs=20, batch_size=4096*16, validation_split=0.2)
history = model.fit(x, y, epochs=20, batch_size=256, validation_split=0.2)

Train on 47520 samples, validate on 11880 samples
Epoch 1/20
47520/47520 [==============================] - 4s 88us/step - loss: 0.6573 - acc: 0.7257 - val_loss: 0.5919 - val_acc: 0.7644
Epoch 2/20
47520/47520 [==============================] - 4s 77us/step - loss: 0.5731 - acc: 0.7649 - val_loss: 0.5503 - val_acc: 0.7775
Epoch 3/20
47520/47520 [==============================] - 4s 77us/step - loss: 0.5303 - acc: 0.7818 - val_loss: 0.5390 - val_acc: 0.7836
Epoch 4/20
47520/47520 [==============================] - 4s 77us/step - loss: 0.5036 - acc: 0.7931 - val_loss: 0.5343 - val_acc: 0.7792
Epoch 5/20
47520/47520 [==============================] - 4s 77us/step - loss: 0.4801 - acc: 0.8007 - val_loss: 0.5153 - val_acc: 0.7916
Epoch 6/20
47520/47520 [==============================] - 4s 77us/step - loss: 0.4603 - acc: 0.8094 - val_loss: 0.5151 - val_acc: 0.7932
Epoch 7/20
47520/47520 [==============================] - 4s 78us/step - loss: 0.4404 - acc: 0.8181 - val_loss: 0.5185 - val_acc

KeyboardInterrupt: ignored

In [0]:

max_score = -1
iter_of_max = 0,0
train_plot = []
val_plot = []
x_axis = []

#Save the weights for k-fold validation
model.save_weights('model.h5')

amount = 120
k_folds = 5

###When i use k_folds instead of 10 for the np.zeros initialization it acts up
val_plot = np.zeros((k_folds,amount))
train_plot = np.zeros((k_folds,amount))
test_plot = np.zeros((k_folds,amount)) 
x_axis = np.zeros((k_folds,amount))
total = int(x.shape[0])
for j in range(k_folds):

    whole = np.arange(0,total)
    test_range = np.arange((j)*(total/k_folds), (j)*(total/k_folds)+(total/k_folds),dtype = 'int')
    
    train_range = np.delete(whole, test_range)
    
    
    #K-fold validation setup
    x_train = x[train_range,:]
    x_pretest = x[test_range,:]
    y_train = y[train_range,:]
    y_pretest = y[test_range,:]
    
    for i in range(amount):

        history = model.fit(x_train, y_train, epochs = 1, batch_size = 4096*16, verbose = 0)
        score = model.evaluate(x_pretest, y_pretest, batch_size = 512, verbose = 0)

        if  float(100 * score[1]) > float(max_score):
            max_score = float(100 * score[1])
            iter_of_max = i,j
        if i%10 == 0:
            print("----------",i,"-",j,"-----------")
            print("Test accuracy: ", (100.0 * score[1]))
            print("Maxscore: ", max_score, "at", iter_of_max,"epoch-kthfold")
        
        #for the plot
        x_axis[j,i] = i
        test_plot[j,i] = (score[1])
        train_plot[j,i] = np.array(history.history['acc'])

        

    if j == k_folds - 1:
        break
    model.load_weights('model.h5')
    print("Reloading Model")
    
print("Done evaluating performance")

---------- 0 - 0 -----------
Test accuracy:  54.23400671393783
Maxscore:  54.23400671393783 at (0, 0) epoch-kthfold
---------- 10 - 0 -----------
Test accuracy:  67.4242424081873
Maxscore:  67.74410774009397 at (9, 0) epoch-kthfold
---------- 20 - 0 -----------
Test accuracy:  71.74242421834155
Maxscore:  71.74242421834155 at (20, 0) epoch-kthfold
---------- 30 - 0 -----------
Test accuracy:  73.41750840546707
Maxscore:  73.80471382478271 at (28, 0) epoch-kthfold
---------- 40 - 0 -----------
Test accuracy:  75.06734007536762
Maxscore:  75.06734007536762 at (40, 0) epoch-kthfold
---------- 50 - 0 -----------
Test accuracy:  75.90067340870095
Maxscore:  75.90067340870095 at (50, 0) epoch-kthfold
---------- 60 - 0 -----------
Test accuracy:  76.82659933462689
Maxscore:  76.95286196088952 at (59, 0) epoch-kthfold
---------- 70 - 0 -----------
Test accuracy:  77.66835018039151
Maxscore:  77.66835018039151 at (70, 0) epoch-kthfold
---------- 80 - 0 -----------
Test accuracy:  77.76094274087

KeyboardInterrupt: ignored